In [1]:
from hdf5storage import loadmat, savemat 
import numpy as np 

from scipy import signal 
from scipy.fftpack import fft

from sklearn.covariance import GraphicalLassoCV

import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__)) 
# make sure it is newer version, works in version 1.2.1

The scikit-learn version is 1.0.2.


In [3]:
# load abs_zscore_all
outdict=loadmat('../Pcorr_single_model/abs_zscore_all.mat')
abs_zscore_all=outdict["abs_zscore_all"] # this file is too large to push to git, please generate it locally by running step 1 in single model

In [4]:
# append abs_zscore from 24 subject together
abs_zscore_subj_L_append =  [[] for i in range(12)]
for ses in range(12):
    for trl in range(12):
        abs_zscore_subj_L_append[ses].append(abs_zscore_all[ses][trl][0]) 

abs_zscore_subj_R_append =  [[] for i in range(12)]
for ses in range(12):
    for trl in range(12):
        abs_zscore_subj_R_append[ses].append(abs_zscore_all[ses][trl][1]) 

abs_zscore_subj_append=[abs_zscore_subj_L_append, abs_zscore_subj_R_append]


In [5]:
ampdata_subj_L =  [[] for i in range(12)]
ampdata_subj_R =  [[] for i in range(12)]
ampdata_subj   =  [ampdata_subj_L, ampdata_subj_R]

In [6]:
# concatenate the trials in each subject
for subj in range(2):
    for ses in range(12):
        ampdata_subj[subj][ses]=np.concatenate(abs_zscore_subj_append[subj][ses])

In [14]:
def partialcorrelation(ampdata,ncv=5):
    pc_lasso = GraphicalLassoCV(cv=ncv)
    nbin = np.shape(ampdata)[1]
    nchan = np.shape(ampdata)[2]
    pcorr = np.zeros((nbin,nchan,nchan))
    alphas = np.zeros(nbin)
    # likelihoods = list()
    likelihoods = np.zeros(nbin)
    covmat = np.zeros((nbin,nchan,nchan))
    for f in range(nbin):
        xx= np.abs(np.squeeze(ampdata[:,f,:]))
        pc_lasso.fit(xx) # the system is too ill for this solver
        covariance = pc_lasso.covariance_
        precision = pc_lasso.precision_
        alphas[f] = pc_lasso.alpha_
        dict_cvresults=pc_lasso.cv_results_
        best_alpha_ind=np.where(dict_cvresults['alphas']==pc_lasso.alpha_)
        # likelihoods.append(pc_lasso.cv_results_["mean_test_score"])
        likelihoods[f] = pc_lasso.cv_results_["mean_test_score"][best_alpha_ind][0]
        parcor = np.zeros((32,32))
        y = np.diag(precision)
        yy = np.outer(y,y)
        yy = np.sqrt(yy)
        parcor = precision/yy
        pcorr[f,:,:] = parcor
        y = np.diag(covariance)
        yy = np.outer(y,y)
        yy = np.sqrt(yy)
        parcor_cov = covariance/yy        
        covmat[f,:,:] = parcor_cov 
    return pcorr,covmat,alphas,likelihoods  


In [15]:
# parameters 
epoch = 1 #units: seconds
maxf = 30 #units: Hz
maxbin = maxf*epoch #convert maxf into number of bins to save

In [ ]:
# fit lasso model for all subjects
pcorr=np.zeros((2,12,maxbin,32,32))
covmat=np.zeros((2,12,maxbin,32,32))
alphas=np.zeros((2,12,maxbin))
likelihoods=np.zeros((2,12,maxbin))
for subj in range(2):
    for ses in range(12):
        ampdata_subj
        pcorr[subj][ses],covmat[subj][ses],alphas[subj][ses],likelihoods[subj][ses] = \
            partialcorrelation(ampdata_subj[subj][ses],ncv=5)

outdict = dict()
outdict['pcorr'] = pcorr
outdict['covmat']= covmat 
outdict['alpha'] = alphas
outdict['likelihood'] = likelihoods

# take 0.5 min

In [ ]:
# save syn_pcorr
savemat('pcorr_subj',outdict,store_python_metadata=True)
